In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType,IntegerType, FloatType
import numpy as np
import pyspark.sql.functions as func
from pyspark.sql.functions import udf

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1558424760139_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


# Stage One

In [2]:
spark = SparkSession \
    .builder \
    .appName("Assignment II") \
    .getOrCreate()

VBox()

In [3]:
rev_data = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"


VBox()

In [4]:
data = spark.read.csv(rev_data,header=True,sep='\t')#.cache()

VBox()

In [5]:
valid_data = data.select('customer_id','review_id','product_id','star_rating','review_body').cache()

VBox()

In [6]:
valid_data.show(5)

VBox()

+-----------+--------------+----------+-----------+--------------------+
|customer_id|     review_id|product_id|star_rating|         review_body|
+-----------+--------------+----------+-----------+--------------------+
|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|          5|Love this CD alon...|
|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|          5|This is the album...|
|   45946560| R9PYL3OYH55QY|B001GCZXW6|          5|  Excellent / thanks|
|   15146326|R3PWBAWUS4NT0Q|B000003EK6|          3|Nice variety of c...|
|   16794688|R15LYP3O51UU9E|B00N1F0BKK|          5|Purchased as a gi...|
+-----------+--------------+----------+-----------+--------------------+
only showing top 5 rows

In [7]:
reviews = valid_data.select('review_id').count()

VBox()

In [8]:
# use distinct function to eliminate duplicate data
users = valid_data.select('customer_id').distinct().count()

VBox()

In [9]:
products = valid_data.select('product_id').distinct().count()

VBox()

In [10]:
print("there are {} reviews of {} different products contributed by {} different users.".format(reviews, products, users))


VBox()

there are 4751577 reviews of 782326 different products contributed by 1940732 different users.

In [11]:
#user_review = valid_data.groupby('customer_id').count().withColumnRenamed('count','reviews_contributed').cache()
user_review = valid_data.groupby('customer_id').count().orderBy(-func.col("count")).cache()


VBox()

In [12]:
# largest number of reviews published by a single user
first_user = user_review.first()
print("largest number of reviews published by a single user is",first_user[1],"published by user",first_user[0])


VBox()

largest number of reviews published by a single user is 7168 published by user 50736950

In [26]:
# the top 10 users ranked by the number of reviews they publish
user_review.show(10)

VBox()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   50736950| 7168|
|   38214553| 5412|
|   51184997| 5369|
|   18116317| 4222|
|   23267387| 4023|
|   50345651| 3793|
|   14539589| 2896|
|   15725862| 2842|
|   19380211| 2592|
|   20018062| 2568|
+-----------+-----+
only showing top 10 rows

In [14]:
# the median number of reviews published by a user
user_median = user_review.approxQuantile("count", [0.5], 0.01)
print("the median number of reviews published by a user is",user_median)

VBox()

the median number of reviews published by a user is [1.0]

In [15]:
#product_review = valid_data.groupby('product_id').count().withColumnRenamed('count','reviews_gained').cache()
product_review = valid_data.groupby('product_id').count().orderBy(-func.col("count")).cache()


VBox()

In [18]:
# the largest number of reviews written for a single product
first_product = product_review.first()
print("the largest number of reviews written for a single product is",first_product[1])

VBox()

the largest number of reviews written for a single product is 3936

In [36]:
# the top 10 products ranked by the number of reviews they have
pr_top_ten = product_review.take(10)
for one in pr_top_ten:
    print("product {} gained {} reviews in total".format(one[0],one[1]))
    
chosen_product = pr_top_ten[-1][0]

VBox()

product B00008OWZG gained 3936 reviews in total
product B0000AGWEC gained 3326 reviews in total
product B00MIA0KGY gained 2699 reviews in total
product B00NEJ7MMI gained 2420 reviews in total
product B000089RVX gained 2376 reviews in total
product B004EBT5CU gained 2106 reviews in total
product B0026P3G12 gained 2080 reviews in total
product B00009PRZF gained 2026 reviews in total
product B00004XONN gained 1901 reviews in total
product B00006J6VG gained 1793 reviews in total

In [20]:
# the median number of reviews a product has
product_median = product_review.approxQuantile("count", [0.5], 0.01)
print("the median number of reviews gained by a product is",product_median)

VBox()

the median number of reviews gained by a product is [2.0]

# Stage Two

##### filtering phase

In [29]:
import re
def sen_seg(text):
    count = 0
    if text is not None:
        sentences = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s",text)
        return len(sentences)    
    else:
        return count

sen_seg_udf = udf(sen_seg, IntegerType())

VBox()

In [30]:
new_valid = valid_data.withColumn("reviews_count", sen_seg_udf('review_body')).filter(func.col("reviews_count") >= 2)


VBox()

In [31]:
# users with less than median number of reviews published
valid_user = user_review.filter(user_review["count"] > user_median[0]).select("customer_id")


VBox()

In [32]:
# filter reviews from products with less than median number of reviews received
valid_product = product_review.filter(product_review["count"] > product_median[0]).select("product_id")

VBox()

In [33]:
# join the three datasets to eliminate invalid data
new_valid = new_valid.join(valid_product, "product_id").join(valid_user, "customer_id").cache()


VBox()

In [34]:
# will take around 4 minutes to get the cache.
# after generatiing caches, subsequent calculations would be much faster 
new_valid.show(5)

VBox()

+-----------+----------+--------------+-----------+--------------------+-------------+
|customer_id|product_id|     review_id|star_rating|         review_body|reviews_count|
+-----------+----------+--------------+-----------+--------------------+-------------+
|   10010810|B007WFQZUA| RQRZQXX7IOMFU|          3|First of all, one...|           10|
|   10010810|B009KT0IE0| R53XG9B98548F|          5|While I can say I...|           10|
|   10071733|B000LP4OII|R2PR1E0IE6173K|          3|I attended the fi...|           29|
|   10071733|B000AARL3Y|R2XDR3IESA0H65|          4|Sony/BMG reached ...|            6|
|   10071733|B000NY166K|R22IB7VR9WVJK6|          5|This collection o...|           15|
+-----------+----------+--------------+-----------+--------------------+-------------+
only showing top 5 rows

##### ranking phase

In [37]:
from pyspark.sql.window import Window

# The customer is segmented by the same id, find the median of the number of reviews included in each customer, 
# and display the top ten customers atfer sorting by medians 
customer_values = new_valid.select("customer_id","reviews_count")
customer_values_window = Window.partitionBy(customer_values.customer_id)
customer_median = func.expr('percentile_approx(reviews_count, 0.5)')

customer_values.withColumn('customer_median_review', customer_median.over(customer_values_window)).drop("reviews_count").distinct().orderBy(func.col("customer_median_review").desc()).limit(10).show()


VBox()

+-----------+----------------------+
|customer_id|customer_median_review|
+-----------+----------------------+
|   37118941|                   229|
|   25628286|                   219|
|   29580246|                   201|
|   23717536|                   190|
|   43879820|                   179|
|   50595705|                   179|
|   15585529|                   177|
|   13255865|                   163|
|   36306364|                   137|
|   19493393|                   136|
+-----------+----------------------+

In [38]:
# The customer is segmented by the same id, find the median of the number of reviews included in each customer, 
# and display the top ten customers after sorting by medians 
product_values = new_valid.select("product_id","reviews_count")
product_values_window = Window.partitionBy(product_values.product_id)
product_median = func.expr('percentile_approx(reviews_count, 0.5)')

product_values.withColumn('product_values_window', product_median.over(product_values_window)).drop("reviews_count").distinct().orderBy(-func.col("product_values_window")).limit(10).show()


VBox()

+----------+---------------------+
|product_id|product_values_window|
+----------+---------------------+
|B00LTQ5EVY|                 1469|
|B009SF2GZU|                  327|
|B000003G29|                  267|
|B000BCH5PK|                  210|
|B0000C0FEW|                  200|
|B00004UFUK|                  163|
|B00AP5M4WM|                  162|
|B004IMODZ4|                  146|
|B00TIPIXIE|                  144|
|B001GM7A6S|                  141|
+----------+---------------------+

# Stage Three


In [39]:
import tensorflow as tf
import tensorflow_hub as hub

# use the chosen product to generate data of positive and negative class
negative_text = valid_data.filter((valid_data.star_rating<=2)&(valid_data.product_id==chosen_product)).select('review_id','review_body')
positive_text = valid_data.filter((valid_data.star_rating>=4)&(valid_data.product_id==chosen_product)).select('review_id','review_body')


VBox()

In [40]:
import re
# to define sentence tokenization method
puncts = [',', '.', '"', ':', ')', '(', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
  '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
  '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

# to split each review into sentences
def senten(row):
    text = row[1]
    sentence = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s",text)
    sentence = [s.replace('<br />','') for s in sentence]
    sentence=[s for s in sentence if s not in puncts]
    sentence = [s.lower().strip() for s in sentence if s!=""]
    return (row[0],sentence)

# to remove punctuation
def remove_punctuation(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, '')
    return x

VBox()

In [41]:
# to transform dataframe into rdd
negative_text_rdd = negative_text.rdd.map(lambda row:(row.review_id,row.review_body))
positive_text_rdd = positive_text.rdd.map(lambda row:(row.review_id,row.review_body))

VBox()

In [42]:
# to split each review into setences and remove none review
rev_clean_text_rdd_nega = negative_text_rdd.map(senten).filter(lambda data: data[1] is not None).cache()
rev_clean_text_rdd_posi = positive_text_rdd.map(senten).filter(lambda data: data[1] is not None).cache()

VBox()

In [43]:
# to tranform the list of sentences into lines
rev_clean_flat_nega = rev_clean_text_rdd_nega.flatMapValues(lambda row :row)
rev_clean_flat_posi = rev_clean_text_rdd_posi.flatMapValues(lambda row :row)

VBox()

In [44]:
def review_embed(rev_text_partition):
    # to get embedding vector from google pretrained model for each sentence
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
    embed = hub.Module(module_url)
    # mapPartition would supply element inside a partition using generator stype
    # this does not fit tensorflow stype
    rev_text_list = [text for text in rev_text_partition]
    print(rev_text_list)
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(embed(rev_text_list))
    return message_embeddings

VBox()

In [45]:
from pyspark.mllib.linalg.distributed import IndexedRowMatrix
from pyspark.mllib.linalg import Vectors

def get_similarity(rev_clean_flat):
    # to compute the similarity of each class
    review_id_rdd = rev_clean_flat.map(lambda row:row[0])
    dims = review_id_rdd.count()
    review_id_text = rev_clean_flat.map(lambda row:row[1])
    review_embedding = review_id_text.mapPartitions(review_embed).cache()
    index_embedding = review_embedding.zipWithIndex()    
    # transform rdd into blockmatrix
    mat = IndexedRowMatrix(index_embedding.map(lambda xi:(xi[1], Vectors.dense(xi[0])))).toBlockMatrix()
    # to compute similarity matrix   
    dot = mat.multiply(mat.transpose())
    mean_sum = dot.blocks.map(lambda row:(row[0],(1-np.round(row[1].toArray(),2)).sum())).map(lambda row : row[1])
    summ = mean_sum.sum()
    return float(summ/((dims -1)*dims))

VBox()

In [46]:
nega_simi = get_similarity(rev_clean_flat_nega)

VBox()

In [47]:
nega_simi

VBox()

0.6684095740731006

In [48]:
posi_simi = get_similarity(rev_clean_flat_posi)

VBox()

In [49]:
posi_simi

VBox()

0.6273248936869297

In [50]:
import itertools
def get_center_and_top_ten(rev_clean_flat):
    # to get the center and top ten sentences
    review_id_rdd = rev_clean_flat.map(lambda row:row[0])
    review_id_text = rev_clean_flat.map(lambda row:row[1])
    review_embedding = review_id_text.mapPartitions(review_embed).cache()
    index_embedding = review_embedding.zipWithIndex()
    # transform rdd into blockmatrix
    mat = IndexedRowMatrix(index_embedding.map(lambda xi:(xi[1], Vectors.dense(xi[0])))).toBlockMatrix()
    # to compute similarity matrix   
    dot = mat.multiply(mat.transpose())
    dot_ori = dot.blocks.map(lambda row:(row[0],(1-np.round(row[1].toArray(),2))))
    mean_dot = dot.blocks.map(lambda row:(row[0],(1-np.round(row[1].toArray(),2).mean(axis=1))))
    mean_sort = mean_dot.sortByKey()
    mean_rdd = mean_sort.map(lambda row: (row[0][0],row[1]))
    shape_list = mean_rdd.groupByKey().map(lambda row:(row[0],np.mean(list(row[1]),axis=0).shape)).collect()
    shape_value_li = []
    shape_value=0
    for i in range(len(shape_list)):
        shape_value_li.append(shape_value)
        shape_value = shape_list[i][1][0]+shape_value
        
    combine_rdd = mean_rdd.groupByKey().map(lambda row:(row[0],np.mean(list(row[1]),axis=0)))
    min_rdd= combine_rdd.map(lambda row:(row[0],np.argmin(row[1]),row[1].min()))
    min_value = min_rdd.map(lambda row:row[2]).min()
    min_result= min_rdd.filter(lambda row:row[2]==min_value)
    min_end = min_result.map(lambda row :(row[0],row[1],row[2],row[1]+shape_value_li[row[0]])).collect()
    review_find = rev_clean_flat.zipWithIndex()
    final_li = []
    for each in min_end:
        re = review_find.filter(lambda row:row[1]==each[3]).collect()[0]
        final_li.append(re)
    dot_tra = dot_ori.filter(lambda row :row[0][0]==min_end[0][0])
    dot_tra_sort = dot_tra.map(lambda row:(row[0],row[1][min_end[0][1],:])).sortByKey()
    # to get the similarity of the centre sentence with other sentences
    one_array = dot_tra_sort.map(lambda row: (row[0][0],row[1])).groupByKey().map(lambda row:(row[0],list(itertools.chain.from_iterable(list(row[1]))))).collect()
    sort_index = np.argsort(one_array[0][1])
    sort_index[:10]
    top_ten = review_find.filter(lambda row:row[1] in sort_index[:10]).collect()
    return final_li, top_ten

VBox()

In [51]:
nega_center, nega_top_ten = get_center_and_top_ten(rev_clean_flat_nega)
# nearly 70secs

VBox()

In [52]:
nega_center

VBox()

[(('R2L4PZC7CHGQ4R', 'this album is much too insipid for people who like to listen to real music.'), 1763)]

In [53]:
nega_top_ten

VBox()

[(('R1CA8OIEZ0B22Y', 'i do like blink 182 but most of this type of this music is junk.'), 251), (('R2UBEJ5JX4PKX1', 'almost everything that is wrong with mainstream music today, is represented in this album.'), 668), (('R195DYX83KWYXU', 'simply put, this album is mediocre at best.'), 671), (('R1DP63VJ4NXY44', 'this album absolutely sucks.'), 1049), (('R1DP63VJ4NXY44', 'this album is full of teeny-bopper pop songs disguised as rock.'), 1052), (('R2L4PZC7CHGQ4R', 'this album is much too insipid for people who like to listen to real music.'), 1763), (('RKN17VFTZQ69P', 'this album is horrid.'), 1802), (('R36BAXRVCR2PFB', "i bet most people who like this album don't even like real punk."), 2295), (('RIQZE81DD7ZM7', "while i haven't heard all of the album, this is the worst band i've heard in a long, long time."), 2342), (('R3QG1GKJO8IL4K', 'this album has in no way let down that genre.'), 2697)]

In [54]:
posi_center, posi_top_ten = get_center_and_top_ten(rev_clean_flat_posi)
# nearly 150secs

VBox()

In [55]:
posi_center

VBox()

[(('R37U2GX41AYFFX', 'this is the ultimate teenage-punk rock cd! every song on the album is awesome and they all put you in a great mood.'), 6266)]

In [56]:
posi_top_ten

VBox()

[(('R26NH9N60MEQ62', 'every song on this album is a gem.'), 267), (('R1LAMC7N0Y96VI', 'this album has some awesome songs.'), 1921), (('R2GCQ3PS817XG9', 'you will seriously love this cd, their single &quot;lifestyles&quot; is not the best song on the cd by far, and every song gets better and better.'), 4506), (('RJ36U6FAGB2X0', 'every song on this album is a great song, performed by gc very well.'), 5277), (('R37U2GX41AYFFX', 'this is the ultimate teenage-punk rock cd! every song on the album is awesome and they all put you in a great mood.'), 6266), (('ROP2FJT2NDY54', 'if you are into the pop-punk sence you will totally love this album! all the songs are super caughty! not only that, the lyrics  are awesome.'), 7223), (('R2UFCYJ6OB21MB', 'this album is such a great follow up to their last cd.'), 7272), (('R1CETHVJRP6CYR', 'this cd is amazing and the songs are so perfect.'), 7312), (('R30ROGEEP9Z4K6', 'i like every song on the cd, and, i really like their first album, i think this one i

# Stage Four

In [57]:
from pyspark.ml.feature import Word2Vec
# word embedding method from pyspark machine learning tools
word2Vec = Word2Vec(vectorSize=10, minCount=0, inputCol="sentence", outputCol="result")


VBox()

In [58]:
def get_similarity_2(rev_clean_flat):
    # to compute the similarity of each class
    df = rev_clean_flat.map(lambda row:(row[0],row[1].split())).toDF()
    df = df.withColumnRenamed('_1','review_id')\
                    .withColumnRenamed('_2','sentence')
    result = word2Vec.fit(df).transform(df)
    result_embedding = result.select("result").rdd.map(lambda row:row[0])
    dims = result_embedding.count()
    index_embedding = result_embedding.zipWithIndex()
    mat = IndexedRowMatrix(index_embedding.map(lambda xi:(xi[1], Vectors.dense(xi[0])))).toBlockMatrix()
    dot = mat.multiply(mat.transpose())
    mean_sum = dot.blocks.map(lambda row:(row[0],(1-np.round(row[1].toArray(),2)).sum())).map(lambda row : row[1])
    summ = mean_sum.sum()
    return float(summ/((dims -1)*dims))

VBox()

In [59]:
nega_simi = get_similarity_2(rev_clean_flat_nega)
posi_simi = get_similarity_2(rev_clean_flat_posi)

VBox()

In [60]:
print(nega_simi)
print(posi_simi)

VBox()

0.9657621852522112
0.9270432099408139

In [61]:
def get_center_and_top_ten_2(rev_clean_flat):
    # to get the center and top ten sentences
    df = rev_clean_flat.map(lambda row:(row[0],row[1].split())).toDF()
    df = df.withColumnRenamed('_1','review_id')\
                    .withColumnRenamed('_2','sentence')
    result = word2Vec.fit(df).transform(df)
    result_embedding = result.select("result").rdd.map(lambda row:row[0])
    index_result_embedding = result_embedding.zipWithIndex()
    # transform rdd into blockmatrix
    mat = IndexedRowMatrix(index_result_embedding.map(lambda xi:(xi[1], Vectors.dense(xi[0])))).toBlockMatrix()
    # to compute similarity matrix 
    dot = mat.multiply(mat.transpose())
    dot_ori = dot.blocks.map(lambda row:(row[0],(1-np.round(row[1].toArray(),2))))
    mean_dot = dot.blocks.map(lambda row:(row[0],(1-np.round(row[1].toArray(),2).mean(axis=1))))
    mean_sort = mean_dot.sortByKey()
    mean_rdd = mean_sort.map(lambda row: (row[0][0],row[1]))
    shape_list = mean_rdd.groupByKey().map(lambda row:(row[0],np.mean(list(row[1]),axis=0).shape)).collect()
    shape_value_li = []
    shape_value=0
    for i in range(len(shape_list)):
        shape_value_li.append(shape_value)
        shape_value = shape_list[i][1][0]+shape_value
        
    combine_rdd = mean_rdd.groupByKey().map(lambda row:(row[0],np.mean(list(row[1]),axis=0)))
    min_rdd= combine_rdd.map(lambda row:(row[0],np.argmin(row[1]),row[1].min()))
    min_value = min_rdd.map(lambda row:row[2]).min()
    min_result= min_rdd.filter(lambda row:row[2]==min_value)
    min_end = min_result.map(lambda row :(row[0],row[1],row[2],row[1]+shape_value_li[row[0]])).collect()
    review_find = rev_clean_flat.zipWithIndex()
    final_li = []
    for each in min_end:
        re = review_find.filter(lambda row:row[1]==each[3]).collect()[0]
        final_li.append(re)
    dot_tra = dot_ori.filter(lambda row :row[0][0]==min_end[0][0])
    dot_tra_sort = dot_tra.map(lambda row:(row[0],row[1][min_end[0][1],:])).sortByKey()
    one_array = dot_tra_sort.map(lambda row: (row[0][0],row[1])).groupByKey().map(lambda row:(row[0],list(itertools.chain.from_iterable(list(row[1]))))).collect()
    # to get the similarity of the centre sentence with other sentences
    sort_index = np.argsort(one_array[0][1])
    sort_index[:10]
    top_ten = review_find.filter(lambda row:row[1] in sort_index[:10]).collect()
    return final_li, top_ten

VBox()

In [62]:
nega_center, nega_top_ten = get_center_and_top_ten_2(rev_clean_flat_nega)

VBox()

In [63]:
nega_center

VBox()

[(('R3TGKXFANRDKW5', 'good charlotte.'), 1293)]

In [64]:
nega_top_ten

VBox()

[(('R1OZJCQTH9OF18', 'sorry, good charlotte.'), 721), (('RJ0PKJ2DZ5TR1', 'but good charlotte?'), 765), (('R3N6ON37QCDDG3', 'good charlotte?'), 885), (('R3TGKXFANRDKW5', 'good charlotte.'), 1293), (('R2JP0QD8H6AVAX', "but good charlotte isn't punk, never will be."), 1360), (('R36BAXRVCR2PFB', 'good charlotte just sucks.'), 2298), (('R3FLONYQEL55MJ', 'i hate good charlotte.'), 2354), (('R10YGSFHSWO4WY', 'and good charlotte'), 2597), (('R39XLG4BT6T792', 'and good charlotte'), 2611), (('R1AV8H99BV69M', "good charlotte isn't origanal."), 2684)]

In [65]:
posi_center, posi_top_ten = get_center_and_top_ten_2(rev_clean_flat_posi)

VBox()

In [66]:
posi_center

VBox()

[(('R3OSCPFNN3SAAS', 'this is it.'), 1165)]

In [67]:
posi_top_ten

VBox()

[(('R2IY8FU1SFU5II', 'i liked this cd.'), 314), (('RCXLMZAFFJCSY', 'i love this cd.'), 1088), (('R3OSCPFNN3SAAS', 'this is it.'), 1165), (('R15LF2TJRWE89', 'i would recommend this cd.'), 1313), (('R31HVXI2N5FV31', 'buy this cd.'), 2697), (('R3CXU9ZFE573IT', 'i highly recommend this cd.'), 4098), (('R27JZ8EENE85GU', 'buy this cd.'), 4248), (('R3KN4RF2NKW2IM', 'i love this cd.'), 4915), (('RWQ1S1163KYQ6', 'i highly recommend this cd.'), 6691), (('R2FY7K8OIYG0E3', 'i love this cd.'), 7918)]